### Cross Validation、last入りデータ、パラメーター調整

In [1]:
# lastとlast_diff を特徴量に追加
# 特徴量選択の数を調査した結果を反映
# パラメーター調整


In [2]:
import gc
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import pandas_profiling as pp

book_name = '832'

warnings.filterwarnings('ignore')
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)



In [3]:
%%time
h5File = "temp_data/h5/feature_select_data_with_last.h5";
key_train = 'feature_select_01/train';
key_valid = 'feature_select_01/valid';
key_test = 'feature_select_01/test';

train = pd.read_hdf(h5File, key_train); gc.collect();
valid = pd.read_hdf(h5File, key_valid); gc.collect();
test = pd.read_hdf(h5File, key_test); gc.collect();


CPU times: user 9.67 s, sys: 15.2 s, total: 24.9 s
Wall time: 31.4 s


72

In [4]:
gc.collect()

0

In [5]:
train

,D_56_avg,S_11_min,B_33_min,D_78_last,D_121_avg,D_122_max,B_10_max,D_48_ldiff,B_19_max,B_22_last,R_10_max,D_52_min,B_11_avg,D_46_max,D_129_max,R_3_last,D_133_last,D_41_max,P_4_max,D_119_max,R_2_max,D_48_avg,R_16_ldiff,D_120_last_0,R_6_max,R_27_min,B_23_last,R_6_last,D_106_avg,D_77_max,D_49_max,R_4_avg,D_112_avg,R_8_avg,D_62_last,D_75_max,D_60_last,D_66_cnt_1,P_2_ldiff,B_14_max,R_5_avg,R_8_max,B_25_avg,B_37_ldiff,D_41_ldiff,S_11_max,B_20_min,S_11_avg,B_12_max,B_38_cnt_2,S_5_last,R_2_ldiff,B_2_ldiff,B_1_min,B_8_avg,B_23_ldiff,R_3_max,S_26_max,D_44_last,D_45_min,D_112_last,D_112_min,R_14_avg,D_60_max,B_5_ldiff,S_22_last,D_78_max,B_18_min,D_132_last,S_16_avg,B_26_max,B_8_max,R_15_max,B_10_avg,D_129_last,S_7_max,B_4_min,D_77_avg,D_54_last,D_46_avg,S_15_last,R_1_avg,R_27_max,D_77_last,B_1_ldiff,D_129_avg,D_54_ldiff,S_3_last,B_1_last,D_84_avg,D_44_max,D_115_ldiff,S_8_max,D_58_last,B_22_avg,D_131_avg,D_43_min,D_74_ldiff,D_110_min,R_11_ldiff,D_59_max,B_2_max,D_41_last,D_91_avg,D_50_max,B_18_last,D_145_ldiff,B_5_min,B_18_max,D_55_ldiff,R_1_last,D_39_avg,S_23_ldiff,D_130_ldiff,D_54_min,S_5_max,R_2_avg,S_8_last,B_33_avg,R_10_avg,R_25_max,D_70_ldiff,D_52_last,B_18_avg,D_119_last,D_46_min,B_30_last_0,B_11_last,D_75_ldiff,D_52_avg,B_14_ldiff,B_5_avg,S_22_max,B_38_last_2,D_121_min,D_41_avg,R_27_avg,D_43_max,D_72_max,B_17_last,D_122_ldiff,B_40_ldiff,B_7_min,D_124_ldiff,B_20_last,B_23_max,P_3_max,B_3_ldiff,D_50_last,D_44_min,B_8_min,D_44_avg,D_58_ldiff,B_16_avg,R_15_ldiff,B_1_avg,S_12_max,B_20_max,S_11_last,D_65_avg,B_1_max,D_42_min,D_80_ldiff,B_2_avg,D_76_last,D_74_last,D_75_avg,B_6_min,R_7_max,D_71_ldiff,B_17_min,B_30_cnt_0,B_4_avg,D_112_ldiff,B_3_max,R_15_avg,B_5_last,D_79_max,D_61_min,D_51_max,B_14_last,B_18_ldiff,S_7_min,R_3_min,B_37_max,D_77_min,S_12_avg,D_55_last,D_59_ldiff,B_28_max,D_55_max,B_5_max,S_7_last,D_62_max,B_38_cnt_4,S_24_max,D_48_min,D_42_max,B_12_last,D_51_last,B_26_last,S_24_last,D_39_last,S_25_last,D_78_ldiff,D_70_last,S_9_avg,D_71_avg,D_139_max,R_6_ldiff,D_39_ldiff,B_9_max,P_4_avg,D_124_max,B_10_min,D_49_min,D_51_avg,B_3_last,D_131_last,S_25_avg,B_39_avg,B_25_last,D_123_avg,D_43_last,R_26_min,D_48_max,D_53_ldiff,B_25_ldiff,D_144_ldiff,B_6_last,D_118_min,B_37_avg,B_6_ldiff,R_3_avg,D_61_max,S_15_max,B_2_last,D_128_last,B_3_min,B_26_ldiff,P_2_avg,R_1_max,R_2_last,P_3_avg,D_129_min,D_44_ldiff,D_129_ldiff,B_8_last,B_21_max,B_7_last,D_46_last,D_50_min,B_9_last,D_62_min,B_29_last,D_74_max,B_21_avg,D_54_avg,D_65_last,S_13_last,B_11_ldiff,D_66_last_null,S_25_ldiff,B_41_avg,D_70_max,D_79_avg,S_7_avg,D_133_ldiff,D_124_last,R_16_avg,B_41_ldiff,D_59_last,D_48_last,S_3_max,B_22_max,B_24_ldiff,S_8_min,S_27_last,S_3_avg,B_20_ldiff,D_49_last,B_7_avg,P_3_last,S_15_avg,R_10_last,B_11_max,P_2_max,S_9_last,D_61_ldiff,D_84_max,D_42_last,D_51_min,B_10_last,D_61_avg,D_55_avg,D_79_ldiff,B_7_ldiff,D_81_last,D_71_min,P_4_min,D_59_avg,D_64_cnt_O,B_9_avg,R_4_max,D_63_cnt_CO,D_56_min,R_11_max,S_6_last,S_22_ldiff,B_9_ldiff,P_3_ldiff,B_19_ldiff,D_74_avg,D_45_max,B_15_last,B_22_ldiff,B_4_ldiff,D_128_min,R_22_ldiff,S_26_avg,B_37_last,D_47_last,B_40_avg,P_2_min,D_78_avg,D_128_avg,D_143_ldiff,S_9_min,D_65_ldiff,S_23_avg,S_8_avg,D_64_last_O,D_58_max,R_11_avg,R_26_max,D_118_avg,D_39_max,R_4_last,B_4_last,B_23_avg,S_23_max,B_7_max,D_111_min,D_43_avg,R_5_last,B_39_max,R_7_ldiff,D_42_avg,R_27_last,R_11_last,B_19_avg,B_41_max,D_56_last,S_12_ldiff,D_45_last,S_15_min,S_3_ldiff,R_7_avg,B_30_cnt_1,D_56_max,R_16_max,R_4_ldiff,D_45_avg,R_10_ldiff,R_1_ldiff,B_19_last,D_62_avg,D_53_max,B_16_last,D_47_min,D_47_max,B_21_ldiff,R_26_avg,D_61_last,D_133_avg,S_23_last,B_40_max,S_3_min,B_26_avg,B_40_last,D_50_avg,B_3_avg,D_120_cnt_0,D_47_avg,D_84_last,B_28_ldiff,D_105_max,B_33_last,B_37_min,D_110_max,R_7_last,D_131_ldiff,B_38_cnt_3,B_11_min,D_141_max,D_75_last,S_24_ldiff,P_3_min,P_2_last,D_84_ldiff,D_128_max,S_2_max_d,B_17_avg,B_36_avg,S_25_min,S_13_max,B_16_max,D_53_min,B_2_min,D_106_min,D_53_last,R_6_avg,R_5_max,B_4_max,D_139_ldiff,D_52_max,B_9_min,B_20_avg,B_13_max,D_79_last

In [6]:
test

,D_56_avg,S_11_min,B_33_min,D_78_last,D_121_avg,D_122_max,B_10_max,D_48_ldiff,B_19_max,B_22_last,R_10_max,D_52_min,B_11_avg,D_46_max,D_129_max,R_3_last,D_133_last,D_41_max,P_4_max,D_119_max,R_2_max,D_48_avg,R_16_ldiff,D_120_last_0,R_6_max,R_27_min,B_23_last,R_6_last,D_106_avg,D_77_max,D_49_max,R_4_avg,D_112_avg,R_8_avg,D_62_last,D_75_max,D_60_last,D_66_cnt_1,P_2_ldiff,B_14_max,R_5_avg,R_8_max,B_25_avg,B_37_ldiff,D_41_ldiff,S_11_max,B_20_min,S_11_avg,B_12_max,B_38_cnt_2,S_5_last,R_2_ldiff,B_2_ldiff,B_1_min,B_8_avg,B_23_ldiff,R_3_max,S_26_max,D_44_last,D_45_min,D_112_last,D_112_min,R_14_avg,D_60_max,B_5_ldiff,S_22_last,D_78_max,B_18_min,D_132_last,S_16_avg,B_26_max,B_8_max,R_15_max,B_10_avg,D_129_last,S_7_max,B_4_min,D_77_avg,D_54_last,D_46_avg,S_15_last,R_1_avg,R_27_max,D_77_last,B_1_ldiff,D_129_avg,D_54_ldiff,S_3_last,B_1_last,D_84_avg,D_44_max,D_115_ldiff,S_8_max,D_58_last,B_22_avg,D_131_avg,D_43_min,D_74_ldiff,D_110_min,R_11_ldiff,D_59_max,B_2_max,D_41_last,D_91_avg,D_50_max,B_18_last,D_145_ldiff,B_5_min,B_18_max,D_55_ldiff,R_1_last,D_39_avg,S_23_ldiff,D_130_ldiff,D_54_min,S_5_max,R_2_avg,S_8_last,B_33_avg,R_10_avg,R_25_max,D_70_ldiff,D_52_last,B_18_avg,D_119_last,D_46_min,B_30_last_0,B_11_last,D_75_ldiff,D_52_avg,B_14_ldiff,B_5_avg,S_22_max,B_38_last_2,D_121_min,D_41_avg,R_27_avg,D_43_max,D_72_max,B_17_last,D_122_ldiff,B_40_ldiff,B_7_min,D_124_ldiff,B_20_last,B_23_max,P_3_max,B_3_ldiff,D_50_last,D_44_min,B_8_min,D_44_avg,D_58_ldiff,B_16_avg,R_15_ldiff,B_1_avg,S_12_max,B_20_max,S_11_last,D_65_avg,B_1_max,D_42_min,D_80_ldiff,B_2_avg,D_76_last,D_74_last,D_75_avg,B_6_min,R_7_max,D_71_ldiff,B_17_min,B_30_cnt_0,B_4_avg,D_112_ldiff,B_3_max,R_15_avg,B_5_last,D_79_max,D_61_min,D_51_max,B_14_last,B_18_ldiff,S_7_min,R_3_min,B_37_max,D_77_min,S_12_avg,D_55_last,D_59_ldiff,B_28_max,D_55_max,B_5_max,S_7_last,D_62_max,B_38_cnt_4,S_24_max,D_48_min,D_42_max,B_12_last,D_51_last,B_26_last,S_24_last,D_39_last,S_25_last,D_78_ldiff,D_70_last,S_9_avg,D_71_avg,D_139_max,R_6_ldiff,D_39_ldiff,B_9_max,P_4_avg,D_124_max,B_10_min,D_49_min,D_51_avg,B_3_last,D_131_last,S_25_avg,B_39_avg,B_25_last,D_123_avg,D_43_last,R_26_min,D_48_max,D_53_ldiff,B_25_ldiff,D_144_ldiff,B_6_last,D_118_min,B_37_avg,B_6_ldiff,R_3_avg,D_61_max,S_15_max,B_2_last,D_128_last,B_3_min,B_26_ldiff,P_2_avg,R_1_max,R_2_last,P_3_avg,D_129_min,D_44_ldiff,D_129_ldiff,B_8_last,B_21_max,B_7_last,D_46_last,D_50_min,B_9_last,D_62_min,B_29_last,D_74_max,B_21_avg,D_54_avg,D_65_last,S_13_last,B_11_ldiff,D_66_last_null,S_25_ldiff,B_41_avg,D_70_max,D_79_avg,S_7_avg,D_133_ldiff,D_124_last,R_16_avg,B_41_ldiff,D_59_last,D_48_last,S_3_max,B_22_max,B_24_ldiff,S_8_min,S_27_last,S_3_avg,B_20_ldiff,D_49_last,B_7_avg,P_3_last,S_15_avg,R_10_last,B_11_max,P_2_max,S_9_last,D_61_ldiff,D_84_max,D_42_last,D_51_min,B_10_last,D_61_avg,D_55_avg,D_79_ldiff,B_7_ldiff,D_81_last,D_71_min,P_4_min,D_59_avg,D_64_cnt_O,B_9_avg,R_4_max,D_63_cnt_CO,D_56_min,R_11_max,S_6_last,S_22_ldiff,B_9_ldiff,P_3_ldiff,B_19_ldiff,D_74_avg,D_45_max,B_15_last,B_22_ldiff,B_4_ldiff,D_128_min,R_22_ldiff,S_26_avg,B_37_last,D_47_last,B_40_avg,P_2_min,D_78_avg,D_128_avg,D_143_ldiff,S_9_min,D_65_ldiff,S_23_avg,S_8_avg,D_64_last_O,D_58_max,R_11_avg,R_26_max,D_118_avg,D_39_max,R_4_last,B_4_last,B_23_avg,S_23_max,B_7_max,D_111_min,D_43_avg,R_5_last,B_39_max,R_7_ldiff,D_42_avg,R_27_last,R_11_last,B_19_avg,B_41_max,D_56_last,S_12_ldiff,D_45_last,S_15_min,S_3_ldiff,R_7_avg,B_30_cnt_1,D_56_max,R_16_max,R_4_ldiff,D_45_avg,R_10_ldiff,R_1_ldiff,B_19_last,D_62_avg,D_53_max,B_16_last,D_47_min,D_47_max,B_21_ldiff,R_26_avg,D_61_last,D_133_avg,S_23_last,B_40_max,S_3_min,B_26_avg,B_40_last,D_50_avg,B_3_avg,D_120_cnt_0,D_47_avg,D_84_last,B_28_ldiff,D_105_max,B_33_last,B_37_min,D_110_max,R_7_last,D_131_ldiff,B_38_cnt_3,B_11_min,D_141_max,D_75_last,S_24_ldiff,P_3_min,P_2_last,D_84_ldiff,D_128_max,S_2_max_d,B_17_avg,B_36_avg,S_25_min,S_13_max,B_16_max,D_53_min,B_2_min,D_106_min,D_53_last,R_6_avg,R_5_max,B_4_max,D_139_ldiff,D_52_max,B_9_min,B_20_avg,B_13_max,D_79_last

In [7]:
valid


customer_ID
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a    0.0
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5    0.0
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1    0.0
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc    0.0
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed    0.0
                                                                   ... 
e0dc8401f13e88ae5fadf2a6cd22ec8f2162a75964a00492ad406dfd437e716f    1.0
e0dcb868e734d5abe60f6a0d9107989aa0cef45c7da5f18642dee0b455c6c311    1.0
e0dcc67c6a5854ab27ac3b799141c2a5c9831471b93a96f1ee62c3e556133956    0.0
e0dd131940680fb03f52d808f3c1308abf4b057188fc0540614d2a543ab8a1ea    0.0
e0dd17ec1fae28e858808935b974ce142d53238d657018a1a2f129335e62d388    1.0
Name: target, Length: 458913, dtype: float64

In [8]:
X_train = train
y_train = valid
X_test = test

In [9]:
gc.collect()


0

In [10]:
%%time
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb

y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'objective': 'binary',
    'max_bin': 500,
    'seed': 55,
    'num_leaves': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_freq': 2,
    'bagging_fraction': 0.8,
    'n_jobs': -1,
    'lambda_l1': 3,
    'lambda_l2': 3,
}

for fold_id, (train_index, test_index) in enumerate(skf.split(X_train, y_train)):
    gc.collect()

    X_tr , X_val = X_train.iloc[train_index], X_train.iloc[test_index]
    y_tr , y_val = y_train.iloc[train_index], y_train.iloc[test_index]

    print("============== fold_id:", fold_id)

    lgb_train = lgb.Dataset(X_tr, y_tr)
    lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

    print("--- train ---")
    %time model = lgb.train(params, lgb_train,\
        valid_sets=[lgb_train, lgb_eval],\
        verbose_eval=10,\
        num_boost_round=1000,\
        early_stopping_rounds=10)

    print("--- oof_train ---")
    %time oof_train[test_index] = model.predict(X_val, num_iteration=model.best_iteration)
    print("--- y_pred ---")
    %time y_pred = model.predict(X_test, num_iteration=model.best_iteration)

    y_preds.append(y_pred)
    models.append(model)


============== fold_id: 0
--- train ---
[LightGBM] [Info] Number of positive: 95062, number of negative: 272068
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 1.315314 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 204168
[LightGBM] [Info] Number of data points in the train set: 367130, number of used features: 423
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.258933 -> initscore=-1.051523
[LightGBM] [Info] Start training from score -1.051523
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.397669	valid_1's binary_logloss: 0.398582
[20]	training's binary_logloss: 0.322266	valid_1's binary_logloss: 0.323888
[30]	training's binary_logloss: 0.282296	valid_1's binary_logloss: 0.284597
[40]	training's binary_logloss: 0.259397	valid_1's binary_logloss: 0.262349
[50]	training's binary_logloss: 0.245322	valid_1's binary_logloss: 0.248851
[60]	training's b

[20]	training's binary_logloss: 0.32205	valid_1's binary_logloss: 0.323627
[30]	training's binary_logloss: 0.282273	valid_1's binary_logloss: 0.284631
[40]	training's binary_logloss: 0.259345	valid_1's binary_logloss: 0.262549
[50]	training's binary_logloss: 0.245232	valid_1's binary_logloss: 0.24918
[60]	training's binary_logloss: 0.236196	valid_1's binary_logloss: 0.240955
[70]	training's binary_logloss: 0.229982	valid_1's binary_logloss: 0.235503
[80]	training's binary_logloss: 0.225541	valid_1's binary_logloss: 0.231813
[90]	training's binary_logloss: 0.222172	valid_1's binary_logloss: 0.22929
[100]	training's binary_logloss: 0.219523	valid_1's binary_logloss: 0.227405
[110]	training's binary_logloss: 0.217242	valid_1's binary_logloss: 0.225926
[120]	training's binary_logloss: 0.215205	valid_1's binary_logloss: 0.224672
[130]	training's binary_logloss: 0.213463	valid_1's binary_logloss: 0.223696
[140]	training's binary_logloss: 0.211853	valid_1's binary_logloss: 0.222863
[150]	trai

[60]	training's binary_logloss: 0.235784	valid_1's binary_logloss: 0.242204
[70]	training's binary_logloss: 0.229603	valid_1's binary_logloss: 0.236883
[80]	training's binary_logloss: 0.225132	valid_1's binary_logloss: 0.233171
[90]	training's binary_logloss: 0.221715	valid_1's binary_logloss: 0.230563
[100]	training's binary_logloss: 0.219039	valid_1's binary_logloss: 0.228663
[110]	training's binary_logloss: 0.216787	valid_1's binary_logloss: 0.227133
[120]	training's binary_logloss: 0.214797	valid_1's binary_logloss: 0.225882
[130]	training's binary_logloss: 0.213022	valid_1's binary_logloss: 0.224838
[140]	training's binary_logloss: 0.21142	valid_1's binary_logloss: 0.223916
[150]	training's binary_logloss: 0.209964	valid_1's binary_logloss: 0.223222
[160]	training's binary_logloss: 0.208613	valid_1's binary_logloss: 0.222685
[170]	training's binary_logloss: 0.207362	valid_1's binary_logloss: 0.222193
[180]	training's binary_logloss: 0.206157	valid_1's binary_logloss: 0.221831
[190

In [11]:
gc.collect()

530

In [12]:
%%time
pd.DataFrame(oof_train).to_csv(f'work/oof_train_kfold_{book_name}.csv', index=False)

scores = [
    m.best_score['valid_1']['binary_logloss'] for m in models
]
score = sum(scores) / len(scores)
print('===CV scores===')
print(scores)
print(score)

===CV scores===
[0.21741547394198416, 0.21999780925496185, 0.2178707554734396, 0.2179624092812563, 0.2191935164940293]
0.2184879928891342
CPU times: user 1.12 s, sys: 70.3 ms, total: 1.19 s
Wall time: 1.24 s


In [13]:
gc.collect()

0

In [14]:
from sklearn.metrics import accuracy_score

y_pred_oof = (oof_train > 0.5).astype(int)
print('accuracy_score:', accuracy_score(y_train, y_pred_oof))


accuracy_score: 0.9032213077424261


In [15]:
len(y_preds)

5

In [16]:
y_preds[0][:10]

array([4.15756403e-02, 8.59389263e-04, 3.98628223e-02, 3.07449562e-01,
       8.64286768e-01, 1.21133649e-03, 9.02875106e-01, 1.19531746e-01,
       6.99979506e-01, 4.04978998e-03])

In [17]:
y_sub = (sum(y_preds) / len(y_preds)).round(2)
# y_sub = (y_sub > 0.5).astype(int)
print(y_sub[:10])


[0.04 0.   0.04 0.27 0.86 0.   0.91 0.18 0.72 0.  ]


In [18]:
sub = pd.read_csv('input/sample_submission.csv')
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0


In [19]:
y_pred

array([0.04330471, 0.00082518, 0.03539883, ..., 0.6766798 , 0.35373249,
       0.06973232])

In [20]:
y_sub

array([0.04, 0.  , 0.04, ..., 0.62, 0.34, 0.07])

In [21]:
# create submission
X_tiny = X_test.iloc[:,[0]]
X_tiny = pd.merge(X_tiny, sub, left_index=True, right_on='customer_ID')
X_tiny['prediction'] = y_sub
X_tiny = X_tiny.drop(columns=X_tiny.columns[[0]])
sub = X_tiny

In [22]:
sub

,customer_ID,prediction
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,0.04
1,00001bf2e77ff879fab36aa4fac689b9ba411dae63ae39...,0.00
2,0000210045da4f81e5f122c6bde5c2a617d03eef67f82c...,0.04
3,00003b41e58ede33b8daf61ab56d9952f17c9ad1c3976c...,0.27
4,00004b22eaeeeb0ec976890c1d9bfc14fd9427e98c4ee9...,0.86
...,...,...
924616,ffff952c631f2c911b8a2a8ca56ea6e656309a83d2f64c...,0.02
924617,ffffcf5df59e5e0bba2a5ac4578a34e2b5aa64a1546cd3...,0.80
924618,ffffd61f098cc056dbd7d2a21380c4804bbfe60856f475...,0.62
924619,ffffddef1fc3643ea179c93245b68dca0f36941cd83977...,0.34


In [23]:
sub.to_csv('output/submission/sub_cross_validation_with_last_feature-select_param-tune.csv', index=False)
